In [1]:
## 找出今天數量，各種均量
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

def combineStr(r):
    stockId, open, close, high, ma1, ma5, ma20, ma60, ma120, ma240 = r["stockId"], r["open"], r["close"], r["high"], r["ma1"], r["ma5"], r["ma20"], r["ma60"], r["ma120"], r["ma240"]
    kbar = 'R' if (close > open) else 'G'
    return f"{stockId},{kbar},{open},{close},{high},{ma1},{ma5},{ma20},{ma60},{ma120},{ma240}"

##
opens    = data.get("price:開盤價").fillna(0).astype(float).tail(1)      #如果資料有空白自動填0
closes   = data.get("price:收盤價").fillna(0).astype(float).tail(1)
highs    = data.get("price:最高價").fillna(0).astype(float).tail(1)
volumns  = data.get("price:成交股數").fillna(0).astype(float)[250: ]  
v = (volumns/1000).round(0).astype(int)
vMa5 = v.rolling(5, min_periods=1).mean().round(0).astype(int).tail(1)
vMa20 = v.rolling(20, min_periods=1).mean().round(0).astype(int).tail(1)
vMa60 = v.rolling(60, min_periods=1).mean().round(0).astype(int).tail(1)
vMa120 = v.rolling(120, min_periods=1).mean().round(0).astype(int).tail(1)
vMa240 = v.rolling(240, min_periods=1).mean().round(0).astype(int).tail(1)

## 本益比、EPS
pER = data.get('price_earning_ratio:本益比')
#print(pER["2328"])

sss = data.get('trading_attention')
# print(sss["2328"])

vdf = v.iloc[-1:].transpose()
vdf["stock"] = vdf.index
vdf.columns = ["ma1", "stockId"]
vdf["ma5"] = vMa5.iloc[-1:].transpose()
vdf["ma20"] = vMa20.iloc[-1:].transpose()
vdf["ma60"] = vMa60.iloc[-1:].transpose()
vdf["ma120"] = vMa120.iloc[-1:].transpose()
vdf["ma240"] = vMa240.iloc[-1:].transpose()
vdf["open"] = opens.iloc[-1:].transpose()
vdf["close"] = closes.iloc[-1:].transpose()
vdf["high"] = highs.iloc[-1:].transpose()
vdf["comp"] = vdf.apply(combineStr, axis = 1)
vdf2 = vdf[ (vdf["stockId"].str.len() == 4)].dropna(axis = 0, how ='any')
# print(vdf2)

s1 = 'id,kbar,yOpen,yClose,yHigh,yVolume,ma5,ma20,ma60,ma120,ma240\n'
for c in vdf2["comp"].tolist():
    s1 += f"{c}\n"
# print(s1)

from datetime import date, timedelta
import time

today = date.today()                        # 取得今天的日期
yesterday = today - timedelta(days=1)       # 計算昨天的日期
yesterday = yesterday.strftime("%Y%m%d")    # 將日期轉換為指定的格式
print(yesterday)
fm.write_LogFile(f"volumeData/ma_{yesterday}.csv", s1)

輸入成功!
20231204


In [55]:
#前置作業準備，只要一次!!
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json

df5k = pd.read_json("paras/mapping5K.json")
dfv = pd.read_csv(f"volumeData/ma_{yesterday}.csv")
dfv["id"] = dfv["id"].astype("string")
# dfv = dfv.drop(columns=['close'])
print(dfv[dfv["id"] == "3265"])

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

url4 = "https://www.wantgoo.com/investrue/all-alive"
r3 = requests.get(url4, headers = headers).content
soup = BeautifulSoup(r3, "html.parser")
rr3 = soup.prettify()
print(rr3)
dfn = pd.read_json(rr3)
dfn = dfn[(dfn["id"]>="1101") & (dfn["id"]<="9999") & (dfn["id"].str.len() == 4)]
dfn = dfn.drop(columns=['type','country','url', 'industries'])
dfn["id"] = dfn["id"].astype("string")

def comp_stockName(r):
    id, name = r["id"], r["name"]
    return f"{id},{name}"

dfn["comp"] = dfn.apply(comp_stockName, axis = 1)
stockNameStr = ""
for c in dfn["comp"].tolist():
    stockNameStr += f"{c}\n"
fm.write_LogFile(f"paras/股票名稱.csv", stockNameStr)  

### 計算量比參數 ###
import time
def getRate5K():
    localtime = time.localtime()
    nowTime = time.strftime("%H:%M", localtime)
    Rate5k = df5k[(df5k["e"] >= nowTime) & (df5k["b"] <= nowTime)]["w"].values[0]  #五分K量比預估放大參數 
    print(f"目前時間：{nowTime},預估量放大係數:{Rate5k}")
    return Rate5k

print(getRate5K())

NameError: name 'yesterday' is not defined

In [53]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.wantgoo.com/investrue/all-quote-info", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
df = pd.read_json(rr)
df = df.drop(columns=['tradeDate', 'time','millionAmount'])
df["id"] = df["id"].astype("string")
df = pd.merge(df, dfn, left_on="id", right_on="id")  ## 結合股票名稱
df["float"] = df["close"].astype("float")
df["amp"] = ((df["close"] - df["previousClose"])/df["previousClose"] * 100).round(2)
df["jump"] = df["open"] - df["previousClose"]
df["jumpRate"] = (df["jump"] / df["open"] * 100).round(2)
df = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["id"].str.len() == 4) & (df["close"] > df["previousClose"])].sort_values("jumpRate", ascending=False)
# df.info()
# print(df.head(50))

r2 = requests.get("https://www.wantgoo.com/stock/all-turnover-rates", headers = headers).content
soup = BeautifulSoup(r2, "html.parser")
rr2 = soup.prettify()
dfa = pd.read_json(rr2)
print(dfv)

dfa["investrueId"] = dfa["investrueId"].astype("string")
dfb = pd.merge(df, dfa, left_on="id", right_on="investrueId")
dfb = dfb.drop(columns=['investrueId'])          
dfb["周轉率"] = dfb["value"].round(2) #周轉率

dfc = pd.merge(dfb, dfv, left_on="id", right_on="id")
dfc["預估量"] = (dfc["volume"] * getRate5K()).astype(int)        ##預估量
dfc["量比"] = (dfc["預估量"] / dfc["yVolume"]).round(2)
dfc["週量比"] = (dfc["預估量"] / dfc["ma5"]).round(2)
dfc["月量比"] = (dfc["預估量"] / dfc["ma20"]).round(2)
dfc["季量比"] = (dfc["預估量"] / dfc["ma60"]).round(2)
dfc["半年量比"] = (dfc["預估量"] / dfc["ma120"]).round(2)
dfc["年量比"] = (dfc["預估量"] / dfc["ma240"]).round(2)
dfc["量比周轉"] = ((dfc["量比"] + dfc["value"]) / 2).round(4)

## 策略1:找出今天開盤跳空
dfc1 = dfc[ (dfc["low"] > dfc["yHigh"]) & (dfc["close"] > dfc["yHigh"])] # & (dfc["close"] >= dfc["open"]) ]  # & (dfc["yVolume"] > 1800) ] 

## 策略2:找出周轉率高  
dfc2 = dfc[ (dfc["yVolume"] > 400) & (dfc["close"] < 400) & (dfc["amp"] > 0.5) & (dfc["amp"] < 2 ) ]  # & (dfc["量比"] > 1) & (dfc["周轉率"] > 1)  ]

df1a = dfc1.loc[:, ["id","market","name","yClose","low","previousClose","open","close","jump","amp","jumpRate","yVolume","預估量","量比","週量比","月量比","季量比","半年量比","年量比","周轉率","量比周轉"]].sort_values("半年量比", ascending=False)
df2a = dfc2.loc[:, ["id","market","name","yClose","low","previousClose","open","close","jump","amp","jumpRate","yVolume","預估量","量比","週量比","月量比","季量比","半年量比","年量比","周轉率","量比周轉"]].sort_values("周轉率", ascending=False)
# display(df1a)
display(df2a)

s1 ,s2 , o_nowTime = '', '', time.strftime("%m%d_%H%M", time.localtime())

df3 = df1a["id"].tolist()
for d in df3:
  s1 += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowTime}_跳空.csv", s1)

df4 = df2a["id"].tolist()
for d in [item for item in df4 if item not in df3]:
  s2 += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowTime}_量比大.csv", s2)


NameError: name 'dfn' is not defined

In [ ]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

## 本益比、EPS
stockId = "2328"
pER = data.get('price_earning_ratio:本益比')
# print(pER[stockId])

EPS = data.get('fundamental_features:每股稅後淨利')
EPS成長率 = EPS.rolling(4).sum()/EPS.shift(4).rolling(4).sum() #最近四季的EPS/最近八季~四季的EPS總和推算
預估EPS = EPS*EPS成長率

print(EPS[stockId].tail(5))
print(EPS成長率[stockId].tail(5))

print(預估EPS[stockId].tail(5))

In [ ]:
## 自己擷取數量，只用來與昨天比較...
import _beowFmt as fm 
dd = dfb[["id","close","volume"]].sort_values("id")
ss = 'id,close,yVolume\n'
for d in dd.values.tolist():
  ss += f"{d[0]},{d[1]},{d[2]}\n"
fm.write_LogFile(f"volumeData/20230110.csv", ss)

# todo ##
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


In [ ]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [6]:
import time

arr = contents.split(",")
arr = [e.replace(".TW", "") for e in arr]
print(arr)

def getKbar(r):
    stockId, open, close = r["id"], r["open"], r["close"]
    return 'R' if (close > open) else 'G'

def getTurnOver(r):
    stockId = r["id"]
    # print((str(stockId) in arr),stockId, arr)
    return 1 if (str(stockId) in arr) else 0 

vdf["kbar"]     = vdf.apply(getKbar, axis = 1)
vdf["turnOver"] = vdf.apply(getTurnOver, axis = 1)
# print(vdf)
vdf2 = vdf[ (vdf["turnOver"] > 0) ].dropna(axis = 0, how ='any')
# print(vdf2)
vdf3 = vdf2[ (vdf["ylow"] > vdf["low"]) & (vdf["close"] > vdf["yclose"]) ].dropna(axis = 0, how ='any')
print(vdf3)

ss , o_nowDate = '', time.strftime("%m%d_1400", time.localtime())
for d in vdf3["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowDate}_破底拉回.csv", ss)

['3479', '6111', '5321', '4744', '4541', '1783', '3324', '1609', '4967', '3017', '5284', '4175', '4164', '8099', '2359', '5356', '2358', '2731', '3664', '5309', '4909', '5353', '6531', '2727', '1762', '2413', '6274', '3207', '3046', '8093', '6241', '4953', '1796', '6125', '5439', '3005', '3305', '3037', '3035', '6223', '3308', '5258', '1776', '6248', '5511', '3454', '2754', '2455', '3011', '3325', '6485', '4915', '2402', '3289', '1316', '2458', '6185', '2707', '6672', '3537', '2515', '8046', '']
          ma1    id   open  close    low   ylow  yclose kbar  turnOver
2359  12277.0  2359  32.85  34.45  32.70  32.75   32.75    R       1.0
3005  17350.0  3005  61.50  62.90  60.20  60.50   61.20    R       1.0
3207   2278.0  3207  75.40  76.70  73.70  75.00   75.00    R       1.0
4541  14391.0  4541  45.10  46.80  44.15  45.30   45.70    R       1.0
8093   1679.0  8093  18.55  18.90  17.65  18.45   18.55    R       1.0


In [1]:
import _beowFmt as fm 
lastDate = fm.getLastFileDate("volumeData", "ma_")

#前置作業準備，只要一次!!
import requests
import pandas as pd
from datetime import datetime
from datetime import date
from bs4 import BeautifulSoup
import json

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180) 

## 取股票名稱 ##
dfStockName = pd.read_csv(f"paras/股票名稱.csv")
dfStockName.columns = ["stockId", "中文名稱"]
dfStockName["stockId"] = dfStockName["stockId"].astype('str')

## 設定AB兩點 ##
dft = pd.read_csv(f"stock-homework/tradingLine.csv")
dft = dft.drop("stockName", axis=1)
dft["stockId"] = dft["stockId"].astype('str')
dft["股票價差"] = dft["after"] - dft["before"]
dft["A_Date"] = pd.to_datetime(dft['beforeD'], format='%Y/%m/%d') #A點
dft["B_Date"] = pd.to_datetime(dft['afterD'], format='%Y/%m/%d')   #B點
dft["C_Date"] = pd.to_datetime(dft["today"], format='%Y/%m/%d')    #C點(模擬今天日)
# dft["C_Date"] = pd.to_datetime(pd.Timestamp.now().date())   #C點(真實今天日)

# print(dft)
dft["BA_diff"] = (dft["B_Date"] - dft["A_Date"]).dt.days.astype(int) / dft["timegap"] * dft["fed"]
dft["CA_diff"] = (dft["C_Date"] - dft["A_Date"]).dt.days.astype(int) / dft["timegap"] * dft["fed"]
dft["C_point"] = dft["before"] + (dft["股票價差"] / dft["BA_diff"] * dft["CA_diff"])
# print(dft.dtypes)


import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
import pandas as pd
import numpy as np
import _beowFmt as fm 

opens    = data.get("price:開盤價").fillna(0).astype(float).tail(1)      #如果資料有空白自動填0
closes   = data.get("price:收盤價").fillna(0).astype(float).tail(1)
highs    = data.get("price:最高價").fillna(0).astype(float).tail(1)
volumns  = data.get("price:成交股數").fillna(0).astype(float)[250: ]  
v        = (volumns/1000).round(0).astype(int)
dfv = v.iloc[-1:].transpose()
dfv["stock"] = dfv.index
dfv.columns = ["ma1", "stockId"]
dfv["open"] = opens.iloc[-1:].transpose()
dfv["close"] = closes.iloc[-1:].transpose()
dfv["high"] = highs.iloc[-1:].transpose()

dfc = pd.merge(dft, dfv, left_on="stockId", right_on="stockId")
dfc = pd.merge(dfc, dfStockName, left_on="stockId", right_on="stockId")
dfc["過高"] = np.where((dfc["close"] > dfc["C_point"]), "V", "")
dfc["K棒壓過"] = np.where((dfc["close"] > dfc["C_point"]) & (dfc["C_point"] > dfc["open"]), "V", "") 
# print(dfc.dtypes)

print(dfc.loc[:, ["timegap","fed","stockId","中文名稱","before","beforeD","after","afterD","C_point","過高", "K棒壓過","open","close","high" ]]) #


最後的檔案日期： 20230706
輸入成功!


 Use "pip install finlab==0.4.5" to update the latest version.


    timegap   fed stockId   中文名稱  before     beforeD   after      afterD     C_point 過高 K棒壓過    open   close    high
0         1  0.71    6188     廣明   57.93   2023/4/12   54.49   2023/5/22   54.232000  V        79.60   79.00   81.50
1         1  0.71    3042     晶技   90.50    2023/3/7   89.50   2023/4/11   88.214286  V        89.90   88.60   89.90
2         5  1.00    2457     飛宏   53.10  2021/12/27   43.00   2022/9/12   40.270270  V        77.50   78.30   80.00
3         5  1.00    2457     飛宏   53.10  2021/12/27   43.65  2022/11/28   41.287500  V        77.50   78.30   80.00
4         5  1.00    3207     耀勝   19.71    2015/3/2   17.18    2018/6/1   14.711811  V        86.80   87.20   88.50
5         5  1.00    4953     緯軟  103.06    2021/4/6   95.05   2022/4/11   88.533757  V       112.50  113.00  116.00
6         5  1.00    3374     精材  180.93   2021/7/12  145.87    2022/6/6  107.080213  V       134.00  134.00  137.50
7         1  0.71    6197    佳必琪   64.90    2023/6/2   63.40   2

Empty DataFrame
Columns: [stockId, 中文名稱, time]
Index: []


In [57]:
from finlab import data
import pandas as pd
import _beowFmt as fm 
import os

rootpath= "D:/project/finlabexportdata"
last1_file_name = fm.getLastFileDate(f"{rootpath}/xq_import", "量價型態_")
last2_file_name = fm.getLast2FileDate(f"{rootpath}/xq_import", "量價型態_")
last_date = last1_file_name.split("_")[0].replace("量價型態", "")
print(last_date)

df1 = pd.read_csv(f"{rootpath}/xq_import/{last1_file_name}.csv")
df2 = pd.read_csv(f"{rootpath}/xq_import/{last2_file_name}.csv")
list1 = df1.columns.tolist()[:-1]
list2 = df2.columns.tolist()[:-1]

diff2 = list(set(list1) - set(list2))

# 將".TW"移出，使用列表解析
stock_diff = [stock.replace('.TW', '') for stock in diff2]

# print(list1)
# print(list2)
# print(diff2)
# print(list1)
# print(stock_diff)

df_diff = pd.DataFrame(stock_diff, columns = ["stockId"])
df_diff["time"] = last1_file_name.removesuffix("_量比大").removeprefix("量價型態2023")

## 取股票名稱 #--------------------------------------------------
dfStockName = pd.read_csv(f"paras/股票名稱.csv")
dfStockName.columns = ["stockId", "中文名稱"]
dfStockName["stockId"] = dfStockName["stockId"].astype('str')
## -------------------------------------------------------------

dfc = pd.merge(df_diff, dfStockName, left_on="stockId", right_on="stockId")
# print(dfc)

csv_file_path = f"{rootpath}/xq_import/周轉率最新_{last_date}.csv"

if not os.path.exists(csv_file_path) or os.path.getsize(csv_file_path) == 0:
  df_combined = pd.DataFrame(columns=['stockId', '中文名稱', 'time']).astype({'stockId': int, '中文名稱': str, 'time': str})
else:
  df_combined = pd.read_csv(csv_file_path, sep='\t')
  # print(df_combined)

df_combined2 = pd.concat([dfc, df_combined], ignore_index=True)
# print(df_combined2)
# 將DataFrame輸出到文字檔
df_combined2.to_csv(csv_file_path, index=False, sep='\t')

最後的檔案日期： 量價型態20230802_0931_量比大
最後的檔案日期： 量價型態20230802_0926_量比大 量價型態20230802_0926_量比大
20230802
  stockId       time 中文名稱
0    5425  0802_0931   台半
1    3017  0802_0931   奇鋐
2    6111  0802_0931  大宇資
3    8358  0802_0931   金居
    stockId       time   中文名稱
0      3580  0802_0926    友威科
1      3306  0802_0926     鼎天
2      6148  0802_0926    驊宏資
3      1905  0802_0926     華紙
4      8016  0802_0926     矽創
5      6874  0802_0926     倍力
6      2913  0802_0926     農林
7      6112  0802_0926    邁達特
8      6470  0802_0926     宇智
9      1591  0802_0926  駿吉-KY
10     2731  0802_0926     雄獅
11     4746  0802_0926     台耀
12     3588  0802_0926     通嘉
13     6829  0802_0926   千附精密
14     3004  0802_0926    豐達科
15     2748  0802_0926     雲品
16     3581  0802_0926     博磊
17     1612  0802_0921     宏泰
18     1618  0802_0921     合機
19     3596  0802_0921     智易
20     3663  0802_0916     鑫科
21     6188  0802_0916     廣明
22     1785  0802_0916    光洋科
23     3044  0802_0916     健鼎
24     1609  0802_0911     

In [27]:
## 整理出唯一的
from finlab import data
import pandas as pd
import _beowFmt as fm 
import os

rootpath= "D:/project/finlabexportdata"
last1_file_name = '周轉率最新_20230816'
df1 = pd.read_csv(f"{rootpath}/xq_import/{last1_file_name}.csv", sep='\t')
df2 = df1.drop_duplicates(subset=['stockId', '中文名稱']).sort_values(by=['time','stockId'], ascending=False)
df3 = df2[['time', 'stockId', '中文名稱']]
print(df3)
df3.to_csv(f"{rootpath}/xq_import/{last1_file_name}a.csv", index=False, sep='\t')

          time  stockId  中文名稱
0    0816_1336     6841  長佳智能
4    0816_1336     6188    廣明
6    0816_1336     6138    茂達
5    0816_1336     6126    信音
1    0816_1336     5439    高技
..         ...      ...   ...
253  0816_0911     3689    湧德
255  0816_0911     3483    力致
254  0816_0911     3306    鼎天
256  0816_0911     2421    建準
257  0816_0911     1795    美時

[100 rows x 3 columns]


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.tpex.org.tw/web/stock/most_active/active_BROKER_vol/rt_brk_result.php?l=zh-tw", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
data = pd.read_json(rr)
data2 = data["aaData"]
# 将每个内部list转换为DataFrame的一行，同时去除首尾空白
df = pd.DataFrame([[item.strip() if isinstance(item, str) else item for item in row] for row in data2],
                  columns=["i", "投信", "買進金額", "賣出金額", "groupId","Stock"])
                  
df["買進金額"] = df["買進金額"].astype(int)
df["賣出金額"] = df["賣出金額"].astype(int)
df[["stockName", "stockId"]] = df["Stock"].str.strip("()").str.split("(", expand=True) # 使用 str.split 将 "Column6" 分割成两列
df["買入金額"] = df["買進金額"] - df["賣出金額"]

df33= df[df["stockId"] == "3483"]
grouped = df.groupby(['stockId', 'stockName'])['買入金額'].sum().reset_index()

sorted_df = grouped.sort_values(by='買入金額', ascending=False)     # 根據買入總金額降序排序

print(sorted_df)


   stockId         stockName  買入金額
9     3363  上詮                 984
21    6188  廣明                 789
4     3213  茂訊                 608
5     3260  威剛                 592
0     1584  精剛                 513
13    4533  協易機                446
26    8069  元太                 425
6     3264  欣銓                 291
15    5009  榮剛                 287
14    4908  前鼎                 282
20    6147  頎邦                 198
10    3483  力致                 185
16    5371  中光電                172
18    6016  康和證                143
2     3163  波若威                109
17    5381  合正                  94
22    6223  旺矽                  80
11    3567  逸昌                   0
12    3693  營邦                 -16
25    8050  廣積                 -65
23    6274  台燿                 -80
27    8086  宏捷科               -157
28    8227  巨有科技              -188
7     3324  雙鴻                -201
3     3178  公準                -209
8     3362  先進光               -239
24    6547  高端疫苗              -526
1     3078  僑威      